In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import pyterrier as pt
import matplotlib.pyplot as plt
import os
# -------------------------------
# Teil 1: Erweiterter Datensatz erstellen
# -------------------------------

DATABASE = "longeval-web"
USER = "dis18"
HOST = "db"
PORT = "5432"
PASSWORD = "dis182425"

engine = create_engine(f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}")

# Dateinamen für den Cache
cache_file_1 = "query_result_1.csv"
cache_file_2 = "query_result_2.csv"

# SQL-Query 1
query = """
SELECT
    DISTINCT "Topic"."queryid" AS ID,
    "Topic"."text_fr" AS Query,
    "Qrel"."docid" AS DocID,
    "Qrel"."relevance" AS Relevanz,
    "Document"."sub_collection" AS Datum,
    "Document"."url"
FROM "Topic"
JOIN "Qrel" ON "Topic"."queryid" = "Qrel"."queryid"
JOIN "Document" ON SUBSTRING("Qrel"."docid", 4) = SUBSTRING("Document"."docid", 4)
WHERE "Qrel"."relevance"::integer > 0
;
"""

# SQL-Query 2
query2 = """
WITH url_counts AS (
    SELECT "url", COUNT(*) AS "Anzahl_Vorkommen"
    FROM "Document"
    GROUP BY "url"
)
SELECT d."url", d."docid", uc."Anzahl_Vorkommen"
FROM "Document" d
JOIN url_counts uc ON d."url" = uc."url"
ORDER BY uc."Anzahl_Vorkommen" DESC
;
"""

# Funktion zum Laden oder Cachen der SQL-Ergebnisse
def load_or_cache_query(query: str, cache_filename: str):
    if os.path.exists(cache_filename):
        print(f"Lade Daten aus Cache: {cache_filename}")
        return pd.read_csv(cache_filename)
    else:
        print(f"Führe SQL-Query aus und speichere nach: {cache_filename}")
        df = pd.read_sql(query, con=engine)
        df.to_csv(cache_filename, index=False)
        return df

# Anwenden der Funktion auf beide Queries
df = load_or_cache_query(query, cache_file_1)
df2 = load_or_cache_query(query2, cache_file_2)

#print(df.head(10))
#print(df2.head(10))

# --
print("SQL")

# Merge der beiden DataFrames anhand der URL
merged_df = pd.merge(df, df2, on='url', how='inner')
# Optional: Umbenennen der Spalte, falls nötig
merged_df = merged_df.rename(columns={'docid': 'docid_x'})







print("Merge DFs")

# Lese die BM25-Ergebnisse aus der CSV-Datei
# csv_df = pd.read_csv("bm25_results.csv")
csv_df = pt.io.read_results("datasets/LongEval-Web/runs/longeval-web-fr-2023-02-BM25.gz")

print("Run eingelesen")

csv_df["docno"] = csv_df["docno"].str.strip("doc")  # the indexed documents prefix the docid with doc, this needs to be removed

print("Docno fix")


# Stelle sicher, dass die Merge-Schlüssel übereinstimmen (Datentypen anpassen)
merged_df['docid_x'] = merged_df['docid_x'].astype(str)
csv_df['docno'] = csv_df['docno'].astype(str)

# Überprüfe zunächst, welche Spalten in csv_df vorhanden sind:
print(csv_df.columns)

# Filtern von csv_df: nur Zeilen, in denen 'rank' numerisch interpretiert (unter 101) ist
csv_df_filtered = csv_df[pd.to_numeric(csv_df['rank'], errors='coerce') < 101]

# Merge der DataFrames mithilfe des gefilterten csv_df
merged_df_final = pd.merge(merged_df, csv_df_filtered, left_on='docid_x', right_on='docno', how='inner')

print(merged_df_final.head(10))

#print(merged_df.head(10))
#print(csv_df.head(10))


# Zweiter Merge: Verbinde den erweiterten Datensatz mit den BM25-Ergebnissen
# merged_df_final = pd.merge(merged_df, csv_df, left_on='docid_x', right_on='docno', how='inner')

print("Datensatz gemerged")

# Speichere den erweiterten Datensatz ab, der die Zeit-Scores enthält
Time_Tags = pd.read_csv('FloAlex/Data/Query_TimeTags.csv')
Join_df = pd.merge(Time_Tags, merged_df_final, on='query', how='inner')

Join_df['Kombinierter_Score'] = (1.1 - (Join_df['Anzahl_Vorkommen'] / 4) * Join_df['score'])/Join_df['tag']*-1



print(Join_df.head())


# Konvertierung zu String
Join_df['qid'] = Join_df['qid'].astype(str)
Join_df['docno'] = Join_df['docno'].astype(str)

qrels_df = Join_df[['qid', 'docno', 'relevanz']].rename(columns={'relevanz': 'label'})
qrels_df['label'] = qrels_df['label'].astype(int)
print(qrels_df.head())

results_df = Join_df[['qid', 'docno', 'Kombinierter_Score']].rename(columns={'Kombinierter_Score': 'score'})

print("Results DataFrame:")
print(results_df.head())
print("\nQrels DataFrame:")
print(qrels_df.head())

evaluation_metrics = ["map", "P_10", "ndcg"]
evaluation_results = pt.Evaluate(results_df, qrels_df, evaluation_metrics)

print("\nEvaluation Results:")
print(evaluation_results)